### PROJETO FINAL



 ALUNOS:




 ESTE PROJETO ...

 - tittle, category:
  - linkar cada notícia a um ID
  - tokenizar 
  - representar de maneira abstrata (BERTimbau) -> Embedding


In [1]:
import os
import joblib
import sys
sys.path.append('..')


import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer

from src.sentence_cleaner import SentenceCleaner
from src.data_scorer_calculator import DataScorerCalculator
from src.sentences_similarity_calculator import SentenceSimilarityCalculator

In [3]:
df_folha = pd.read_csv("../dataset/FolhaArticles.csv", low_memory=False, sep="\t", encoding="UTF-8")
df_folha['publication_date'] = pd.to_datetime(df_folha['Published'], errors='coerce')
df_folha = df_folha.dropna(subset=['Title','publication_date'])
df_folha = df_folha.drop_duplicates(subset=['Title'])

In [27]:
df_g1 = pd.read_csv("../dataset/G1Articles.csv", low_memory=False, encoding="UTF-8")

In [35]:
df_g1

,category,image,link,publication_date,source,text,title
0,Piracicaba e Região,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/sp/piracicaba-regiao/noti...,2024-05-02 00:40:07.369630,G1,"Segundo especialista, registro ocorre no prime...",Quem tem registro de arcada dentária e como el...
1,São Paulo,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/sp/sao-paulo/noticia/2024...,2024-05-02 00:40:07.393406,G1,Proposta que prevê adesão da capital foi aprov...,Câmara de SP faz votação final do projeto de l...
2,Mundo,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/mundo/noticia/2024/05/02/...,2024-05-02 00:40:07.398813,G1,Centenas de pessoas foram presas e as faculdad...,O que querem estudantes que protestam contra g...
3,Rio Grande do Sul,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/rs/rio-grande-do-sul/noti...,2024-05-02 00:40:07.408700,G1,"Ao todo, 114 municípios do Rio Grande do Sul f...",Mais cidades do RS sofrem impactos de temporal...
4,Rio Grande do Sul,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/rs/rio-grande-do-sul/noti...,2024-05-02 00:40:07.417674,G1,Previsão para os próximos dias alerta para chu...,"Temporais no RS: em quatro dias, estado tem o ..."
...,...,...,...,...,...,...,...
66962,Roraima,https://s2-g1.glbimg.com/Un-WCAkGH2DgdaRvm_sqV...,https://g1.globo.com/rr/roraima/noticia/2024/0...,Há 1 mês,G1,Cobra foi encontrada por um casal de agriculto...,VÍDEO: Sucuri-verde de cerca de 4 metros é enc...
66963,Mundo,https://s2-g1.glbimg.com/Un-WCAkGH2DgdaRvm_sqV...,https://g1.globo.com/mundo/noticia/2024/03/31/...,Há 1 mês,G1,"Neste domingo (31), milhares protestaram em Je...",Netanyahu diz que Hamas 'endurece' negociação ...
66964,Tocantins,https://s2-g1.glbimg.com/Un-WCAkGH2DgdaRvm_sqV...,https://g1.globo.com/to/tocantins/noticia/2024...,Há 1 mês,G1,Vítima foi identificada como Marcilene Alcânta...,Jovem assassinada a facadas em bar tinha medid...
66965,Sul de Minas,https://s2-g1.glbimg.com/Un-WCAkGH2DgdaRvm_sqV...,https://g1.globo.com/mg/sul-de-minas/noticia/2...,Há 1 mês,G1,Suspeitos estavam em outra motocicleta. Eles f...,Jovem de 21 anos é baleado enquanto andava de ...


In [34]:
df_g1.dropna(subset=['link'])

,category,image,link,publication_date,source,text,title
0,Piracicaba e Região,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/sp/piracicaba-regiao/noti...,2024-05-02 00:40:07.369630,G1,"Segundo especialista, registro ocorre no prime...",Quem tem registro de arcada dentária e como el...
1,São Paulo,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/sp/sao-paulo/noticia/2024...,2024-05-02 00:40:07.393406,G1,Proposta que prevê adesão da capital foi aprov...,Câmara de SP faz votação final do projeto de l...
2,Mundo,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/mundo/noticia/2024/05/02/...,2024-05-02 00:40:07.398813,G1,Centenas de pessoas foram presas e as faculdad...,O que querem estudantes que protestam contra g...
3,Rio Grande do Sul,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/rs/rio-grande-do-sul/noti...,2024-05-02 00:40:07.408700,G1,"Ao todo, 114 municípios do Rio Grande do Sul f...",Mais cidades do RS sofrem impactos de temporal...
4,Rio Grande do Sul,https://s2-g1.glbimg.com/QlaomfaCOFGJvvta1G2dR...,https://g1.globo.com/rs/rio-grande-do-sul/noti...,2024-05-02 00:40:07.417674,G1,Previsão para os próximos dias alerta para chu...,"Temporais no RS: em quatro dias, estado tem o ..."
...,...,...,...,...,...,...,...
66961,Blog do Mauro Ferreira,https://s2-g1.glbimg.com/Un-WCAkGH2DgdaRvm_sqV...,https://g1.globo.com/pop-arte/musica/blog/maur...,Há 1 mês,G1,NaN,"Caim, cantor paulista que morreu aos 80 anos, ..."
66962,Roraima,https://s2-g1.glbimg.com/Un-WCAkGH2DgdaRvm_sqV...,https://g1.globo.com/rr/roraima/noticia/2024/0...,Há 1 mês,G1,Cobra foi encontrada por um casal de agriculto...,VÍDEO: Sucuri-verde de cerca de 4 metros é enc...
66963,Mundo,https://s2-g1.glbimg.com/Un-WCAkGH2DgdaRvm_sqV...,https://g1.globo.com/mundo/noticia/2024/03/31/...,Há 1 mês,G1,"Neste domingo (31), milhares protestaram em Je...",Netanyahu diz que Hamas 'endurece' negociação ...
66964,Tocantins,https://s2-g1.glbimg.com/Un-WCAkGH2DgdaRvm_sqV...,https://g1.globo.com/to/tocantins/noticia/2024...,Há 1 mês,G1,Vítima foi identificada como Marcilene Alcânta...,Jovem assassinada a facadas em bar tinha medid...


In [ ]:
data_scorer_calculator = DataScorerCalculator()

In [4]:
df_folha['normalized_data_scores'] = data_scorer_calculator.calculate_data_score(df_folha['publication_date'])

In [5]:
df_folha

,Title,Content,Url,Published,categories,publication_date,normalized_data_scores
0,Visões da batalha na Rússia e no Ocidente são ...,Quatro décadas de Guerra Fria e o renovado con...,https://www1.folha.uol.com.br/mundo/2019/06/vi...,2019-06-06 02:00:00.0000000,mundo,2019-06-06 02:00:00,0.129192
1,Bolsonaro é intimado pela PF sobre golpe e pod...,A Polícia Federal intimou Jair Bolsonaro (PL) ...,https://www1.folha.uol.com.br/poder/2024/02/pf...,2024-02-19 12:45:00.0000000,poder,2024-02-19 12:45:00,0.000225
2,Hayao Miyazaki passa a vida a limpo em O Menin...,Como vocês vivem? A pergunta parece ser feita ...,https://www1.folha.uol.com.br/ilustrada/2024/0...,2024-02-19 07:00:00.0000000,ilustrada,2024-02-19 07:00:00,0.000225
3,Série produzida pelo Estúdio Folha ganha Prêmi...,"A série ""Caminhos Proibidos"", produzida pelo E...",https://estudio.folha.uol.com.br/estudio/2024/...,2024-01-18 16:49:00.0000000,estudio,2024-01-18 16:49:00,0.002626
4,"Como é 'Crystal', do Cirque du Soleil, que vem...",Uma mulher expressa sua angústia dançando sobr...,https://www1.folha.uol.com.br/ilustrada/2024/0...,2024-02-19 15:15:00.0000000,ilustrada,2024-02-19 15:15:00,0.000225
...,...,...,...,...,...,...,...
192943,Não chegou o momento de mexer no teto - 08/09/...,"Fabio Giambiagi e Guilherme Tinoco, em texto p...",https://www1.folha.uol.com.br/colunas/samuelpe...,2019-09-08 02:00:00.0000000,colunas,2019-09-08 02:00:00,0.122140
192951,Saúde masculina é negligenciada desde a infânc...,"No Brasil, a expectativa de vida dos homens é ...",https://www1.folha.uol.com.br/equilibrioesaude...,2022-09-22 18:57:00.0000000,equilibrioesaude,2022-09-22 18:57:00,0.038863
192954,Modernização da agricultura esbarra em escasse...,Até onde vai a evolução tecnológica da agricul...,https://www1.folha.uol.com.br/seminariosfolha/...,2018-03-22 02:00:00.0000000,seminariosfolha,2018-03-22 02:00:00,0.162278
192966,Forças Armadas da Venezuela se declaram 'em al...,"As Forças Armadas venezuelanas se declararam, ...",https://www1.folha.uol.com.br/mundo/2019/02/fo...,2019-02-19 15:41:00.0000000,mundo,2019-02-19 15:41:00,0.137220


##### Testar utilizando a coluna categories
 - remover categories insignificantes <= 10
 - adicionar ao titulo com a categoria repetida para ter mais peso

In [6]:
data_train = df_folha[['Title','Content', 'normalized_data_scores']].copy()

print('Training dataset size:', len(data_train))

Training dataset size: 166397


In [7]:
data_train.loc[:, 'text_full']= data_train.loc[:, 'Title'].str.cat(data_train['Content'], sep=' ')

In [8]:
data_train

,Title,Content,normalized_data_scores,text_full
0,Visões da batalha na Rússia e no Ocidente são ...,Quatro décadas de Guerra Fria e o renovado con...,0.129192,Visões da batalha na Rússia e no Ocidente são ...
1,Bolsonaro é intimado pela PF sobre golpe e pod...,A Polícia Federal intimou Jair Bolsonaro (PL) ...,0.000225,Bolsonaro é intimado pela PF sobre golpe e pod...
2,Hayao Miyazaki passa a vida a limpo em O Menin...,Como vocês vivem? A pergunta parece ser feita ...,0.000225,Hayao Miyazaki passa a vida a limpo em O Menin...
3,Série produzida pelo Estúdio Folha ganha Prêmi...,"A série ""Caminhos Proibidos"", produzida pelo E...",0.002626,Série produzida pelo Estúdio Folha ganha Prêmi...
4,"Como é 'Crystal', do Cirque du Soleil, que vem...",Uma mulher expressa sua angústia dançando sobr...,0.000225,"Como é 'Crystal', do Cirque du Soleil, que vem..."
...,...,...,...,...
192943,Não chegou o momento de mexer no teto - 08/09/...,"Fabio Giambiagi e Guilherme Tinoco, em texto p...",0.122140,Não chegou o momento de mexer no teto - 08/09/...
192951,Saúde masculina é negligenciada desde a infânc...,"No Brasil, a expectativa de vida dos homens é ...",0.038863,Saúde masculina é negligenciada desde a infânc...
192954,Modernização da agricultura esbarra em escasse...,Até onde vai a evolução tecnológica da agricul...,0.162278,Modernização da agricultura esbarra em escasse...
192966,Forças Armadas da Venezuela se declaram 'em al...,"As Forças Armadas venezuelanas se declararam, ...",0.137220,Forças Armadas da Venezuela se declaram 'em al...


#### Limpeza e tratamento das sentenças

In [9]:
sentece_cleaner = SentenceCleaner()

In [10]:
data_train.loc[:, 'sentences'] = data_train.loc[:, 'text_full'].apply(sentece_cleaner.clear_sentence)

In [11]:
data_train

,Title,Content,normalized_data_scores,text_full,sentences
0,Visões da batalha na Rússia e no Ocidente são ...,Quatro décadas de Guerra Fria e o renovado con...,0.129192,Visões da batalha na Rússia e no Ocidente são ...,Visões batalha Rússia Ocidente inconciliáveis ...
1,Bolsonaro é intimado pela PF sobre golpe e pod...,A Polícia Federal intimou Jair Bolsonaro (PL) ...,0.000225,Bolsonaro é intimado pela PF sobre golpe e pod...,Bolsonaro intimado PF golpe depor Poder A Pol...
2,Hayao Miyazaki passa a vida a limpo em O Menin...,Como vocês vivem? A pergunta parece ser feita ...,0.000225,Hayao Miyazaki passa a vida a limpo em O Menin...,Hayao Miyazaki passa vida limpo O Menino Garça...
3,Série produzida pelo Estúdio Folha ganha Prêmi...,"A série ""Caminhos Proibidos"", produzida pelo E...",0.002626,Série produzida pelo Estúdio Folha ganha Prêmi...,Série produzida Estúdio ganha Prêmio Nacional ...
4,"Como é 'Crystal', do Cirque du Soleil, que vem...",Uma mulher expressa sua angústia dançando sobr...,0.000225,"Como é 'Crystal', do Cirque du Soleil, que vem...",Como Crystal Cirque du Soleil Brasil Ilustrad...
...,...,...,...,...,...
192943,Não chegou o momento de mexer no teto - 08/09/...,"Fabio Giambiagi e Guilherme Tinoco, em texto p...",0.122140,Não chegou o momento de mexer no teto - 08/09/...,Não chegou mexer teto Samuel Pessôa Fabio Gia...
192951,Saúde masculina é negligenciada desde a infânc...,"No Brasil, a expectativa de vida dos homens é ...",0.038863,Saúde masculina é negligenciada desde a infânc...,Saúde masculina negligenciada infância Equilí...
192954,Modernização da agricultura esbarra em escasse...,Até onde vai a evolução tecnológica da agricul...,0.162278,Modernização da agricultura esbarra em escasse...,Modernização agricultura esbarra escassez ...
192966,Forças Armadas da Venezuela se declaram 'em al...,"As Forças Armadas venezuelanas se declararam, ...",0.137220,Forças Armadas da Venezuela se declaram 'em al...,Forças Armadas Venezuela declaram alerta fecha...


In [12]:
data_train['sentences'].values

array(['Visões batalha Rússia Ocidente inconciliáveis hoje  Mundo Quatro décadas Guerra Fria renovado conflito Ocidente Rússia tornaram inconciliáveis visões vitoriosos Segunda Guerra Mundial dia    Aos olhos ocidentais Dia D selou começo Alemanha nazista ajudou evitar Europa caísse mãos União Soviética    Para russos ação secundária “ Não impacto decisivo guerra Isso havia sido determinado vitórias Exército Vermelho ” disse evento  porta-voz Ministério Relações Exteriores russo Maria Zakharova   Ela replica visão verdadeiro golpe mortal forças Adolf Hitler épicos embates ocorridos   Frente Oriental      Tropas americanas desembarcam praia Omaha Normandia - Robert F. Sargent U.S. National Archives Reuters“Logo º aniversário Dia D Normandia  representei forças americanas conferência governo Praga ‘ A Abertura Segunda Frente Europa ’ Isso tendo tchecos sido ocupados soviéticos sido comunistas  ” conta Bryan van Sweringen   Professor história Universidade Livre Berlim americano Van Swerin

In [13]:
tfidf_vectorizer = TfidfVectorizer()

In [14]:
tfidf_matrix = tfidf_vectorizer.fit_transform(data_train['sentences'].values)

In [15]:
tfidf_matrix

<166397x472125 sparse matrix of type '<class 'numpy.float64'>'
	with 44190367 stored elements in Compressed Sparse Row format>

In [16]:
sentences_sym_calculator = SentenceSimilarityCalculator()

In [38]:
raw_sentence = 'Trump diz que foi “lindo ver” ação policial contra manifestantes pró-Palestina'
new_doc = sentece_cleaner.clear_sentence(raw_sentence)
new_sent = tfidf_vectorizer.transform([new_doc])

In [40]:
print(f'Alvo -> {raw_sentence}')
print('\nTop-5 similarity rate:\n')

for idx in sentences_sym_calculator.sym_by_date(new_sent, tfidf_matrix, data_train['normalized_data_scores'].values, 70, 5):
    print(f'{data_train["Title"].values[idx]}')

Alvo -> Trump diz que foi “lindo ver” ação policial contra manifestantes pró-Palestina

Top-5 similarity rate:

Trump será julgado em março por pagamentos a atriz pornô - 15/02/2024 - Mundo - Folha
Lula se encontra com premiê da Autoridade Palestina - 17/02/2024 - Mundo - Folha
Fala de Lula sobre Holocausto pode enfraquecer governo - 18/02/2024 - Mundo - Folha
EUA: Idade prejudica mais Biden do que Trump - 13/02/2024 - Mundo - Folha
Trump vê suas décadas de impunidade chegarem ao fim - 07/02/2024 - Lúcia Guimarães - Folha
